In [1]:
import pandas as pd
import os

In [2]:
log_data = pd.read_json('data/log_data/2018/11/2018-11-02-events.json', lines=True)

path = ['A', 'B']
path_2 = ['A', 'B', 'C']

song_data = pd.DataFrame({'0':['0']})
for i in path:
    for j in path_2:
        for p in os.listdir(f'data/song_data/A/{i}/{j}/'):
            if song_data.columns[0] == '0':
                song_data = pd.read_json(f'data/song_data/A/{i}/{j}/{p}', lines=True)
            else:
                song_data = pd.concat([song_data, pd.read_json(f'data/song_data/A/{i}/{j}/{p}', lines=True)])

In [3]:
import pandas as pd
import psycopg2
from sql_queries import create_table_queries, drop_table_queries

conn = psycopg2.connect(database="postgres", user='postgres', password='0908', host='127.0.0.1', port= '5432')
conn.set_session(autocommit=True)
cursor = conn.cursor()
cursor.execute('''DROP DATABASE IF EXISTS sparkifydb''')
cursor.execute('''CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0''')

conn.close()

conn = psycopg2.connect(database="sparkifydb", user='postgres', password='0908', host='127.0.0.1', port= '5432')
cursor = conn.cursor()

for command in drop_table_queries:
    cursor.execute(command)
    conn.commit()
    
for command in create_table_queries:
    cursor.execute(command)
    conn.commit()

In [4]:
log_data.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.9922,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101


### User

In [5]:
user_df = log_data[['userId','firstName','lastName','gender','level']]
# user_df['userId'] = user_df['userId'].astype('int')


### Song

In [6]:
song_df = song_data[['song_id','title','artist_id','year','duration']]
song_df['year'] = song_df['year'].astype('int')
song_df['duration'] = song_df['duration'].astype('int')


C:\Users\ASUS ZENBOOK\AppData\Local\Temp\ipykernel_14844\1942188612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['year'] = song_df['year'].astype('int')
C:\Users\ASUS ZENBOOK\AppData\Local\Temp\ipykernel_14844\1942188612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['duration'] = song_df['duration'].astype('int')


### Artist

In [7]:
artist_df = song_data[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

In [35]:
user_df['userId'].iloc[0].astype('int').dtype

dtype('int32')

### Time

In [8]:
time_df = pd.DataFrame({'start_time': log_data['ts']})
time_df['year'] = pd.to_datetime(log_data['ts'], unit='ms').dt.year.astype('int')
time_df['month'] = pd.to_datetime(log_data['ts'], unit='ms').dt.month.astype('int')
time_df['week'] = pd.to_datetime(log_data['ts'], unit='ms').dt.isocalendar().week.astype('int')
time_df['day'] = pd.to_datetime(log_data['ts'], unit='ms').dt.day.astype('int')
time_df['hour'] = pd.to_datetime(log_data['ts'], unit='ms').dt.hour.astype('int')
time_df['weekday'] = pd.to_datetime(log_data['ts'], unit='ms').dt.weekday.astype('int')

time_df = time_df[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]

In [9]:
time_df.head(1)

,start_time,hour,day,week,month,year,weekday
0,1541121934796,1,2,44,11,2018,4


### Songplays

In [27]:
songplay = log_data[['ts']]
for i in ['userId', 'level', 'song', 'artist', 'sessionId', 'location', 'userAgent', 'length']:
    if log_data[i].dtype == 'int64':
        songplay[i] = log_data[i].astype('int').values
    else:
        songplay[i] = log_data[i].values


C:\Users\ASUS ZENBOOK\AppData\Local\Temp\ipykernel_14844\1179505022.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songplay[i] = log_data[i].astype('int').values
C:\Users\ASUS ZENBOOK\AppData\Local\Temp\ipykernel_14844\1179505022.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songplay[i] = log_data[i].values
C:\Users\ASUS ZENBOOK\AppData\Local\Temp\ipykernel_14844\1179505022.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

## Insert

In [11]:
from sql_queries import insert_table_artists, insert_table_songs, insert_table_songplays, insert_table_time, insert_table_users

### Time

In [12]:
for i in range(len(time_df)):
    cursor.execute(insert_table_time, time_df.iloc[i].to_list())

In [13]:
pd.read_sql('select * from time', conn)

c:\Users\ASUS ZENBOOK\miniconda3\envs\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,start_time,hour,day,week,month,year,weekday
0,1541121934796,1,2,44,11,2018,4
1,1541122176796,1,2,44,11,2018,4
2,1541122241796,1,2,44,11,2018,4
3,1541122457796,1,2,44,11,2018,4
4,1541126568796,2,2,44,11,2018,4
...,...,...,...,...,...,...,...
166,1541184339796,18,2,44,11,2018,4
167,1541187675796,19,2,44,11,2018,4
168,1541191023796,20,2,44,11,2018,4
169,1541191251796,20,2,44,11,2018,4


### User

In [14]:
for i in range(len(user_df)):
    cursor.execute(insert_table_users, user_df[['userId']].iloc[i].to_list()+user_df[['firstName','lastName','gender','level']].iloc[i].to_list())


In [15]:
pd.read_sql('select * from users', conn)


c:\Users\ASUS ZENBOOK\miniconda3\envs\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,user_id,first_name,last_name,gender,level
0,101,Jayden,Fox,M,free
1,83,Stefany,White,F,free
2,66,Kevin,Arellano,M,free
3,48,Marina,Sutton,F,free
4,86,Aiden,Hess,M,free
5,17,Makinley,Jones,F,free
6,15,Lily,Koch,F,paid
7,89,Kynnedi,Sanchez,F,free
8,80,Tegan,Levine,F,free
9,44,Aleena,Kirby,F,paid


### Song

In [16]:
for i in range(len(song_df)):
    cursor.execute(insert_table_songs, song_df[['song_id','title','artist_id']].iloc[i].to_list()+song_df[['year','duration']].iloc[i].to_list())


In [17]:
pd.read_sql('select * from songs', conn)


c:\Users\ASUS ZENBOOK\miniconda3\envs\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.0
1,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.0
2,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.0
3,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.0
4,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.0
...,...,...,...,...,...
66,SOINLJW12A8C13314C,City Slickers,AR8IEZO1187B99055E,2008,149.0
67,SOGDBUF12A8C140FAA,Intro,AR558FS1187FB45658,2003,75.0
68,SORRZGD12A6310DBC3,Harajuku Girls,ARVBRGZ1187FB4675A,2004,290.0
69,SODAUVL12A8C13D184,Prognosis,ARWB3G61187FB49404,2000,363.0


### Artist

In [18]:
for i in range(len(artist_df)):
    cursor.execute(insert_table_artists, artist_df.iloc[i].to_list())


In [19]:
pd.read_sql('select * from artists', conn)


c:\Users\ASUS ZENBOOK\miniconda3\envs\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,artist_id,name,location,latitude,longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
1,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
2,ARKRRTF1187B9984DA,Sonora Santanera,,NaN,NaN
3,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
4,ARXR32B1187FB57099,Gob,,NaN,NaN
...,...,...,...,...,...
64,AR8IEZO1187B99055E,Marc Shaiman,,NaN,NaN
65,AR558FS1187FB45658,40 Grit,,NaN,NaN
66,ARVBRGZ1187FB4675A,Gwen Stefani,,NaN,NaN
67,ARWB3G61187FB49404,Steve Morse,"Hamilton, Ohio",NaN,NaN


### Songplay

In [20]:
title = 'a'
artist_name = 'b'
duration = 1

pd.read_sql(f"""select song_id, artists.artist_id from songs 
    JOIN artists ON songs.artist_id = artists.artist_id
    WHERE songs.artist_id = artists.artist_id 
    and title='{title}'
    and artists.name = '{artist_name}' 
    and songs.duration = {duration}""" , conn)

c:\Users\ASUS ZENBOOK\miniconda3\envs\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,song_id,artist_id


In [21]:
query = """select song_id, artists.artist_id from songs 
    JOIN artists ON songs.artist_id = artists.artist_id
    WHERE songs.artist_id = artists.artist_id 
    and title = %s
    and artists.name = %s 
    and songs.duration = %s"""

In [28]:
results = cursor.execute(query, (title, artist_name, duration))
song_id, artist_id = results if results else None, None

In [23]:
songplay.head(1)

,ts,userId,level,song,artist,sessionId,location,userAgent
0,1541121934796,101,free,Am I High (Feat. Malice),N.E.R.D. FEATURING MALICE,184,"New Orleans-Metairie, LA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."


In [29]:
for i in range(len(songplay)):
    title = songplay['song'].to_list()[i]
    artist_name = songplay['artist'].to_list()[i]
    duration = songplay['length'].to_list()[i]

    results = cursor.execute(query, (title, artist_name, duration))
    song_id, artist_id = results if results else None, None

    cursor.execute(insert_table_songplays, songplay[['ts']].iloc[i].astype('int64').to_list() \
        + songplay[['userId']].iloc[i].astype('int').to_list() \
        + songplay[['level']].iloc[i].to_list() \
        + [song_id, artist_id]\
        + songplay[['sessionId']].iloc[i].astype('int').to_list() \
        + songplay[['location', 'userAgent']].iloc[i].to_list())


In [30]:
pd.read_sql('select * from songplays', conn)


c:\Users\ASUS ZENBOOK\miniconda3\envs\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,1,1541121934796,101,free,None,None,184,"New Orleans-Metairie, LA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
1,2,1541122176796,83,free,None,None,82,"Lubbock, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,3,1541122241796,83,free,None,None,82,"Lubbock, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,4,1541122457796,83,free,None,None,82,"Lubbock, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,5,1541126568796,66,free,None,None,153,"Harrisburg-Carlisle, PA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
...,...,...,...,...,...,...,...,...,...
337,338,1541184339796,71,free,None,None,70,"Columbia, SC","""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like..."
338,339,1541187675796,54,free,None,None,53,"Yuba City, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...
339,340,1541191023796,3,free,None,None,112,"Saginaw, MI",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...
340,341,1541191251796,3,free,None,None,112,"Saginaw, MI",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...


In [1]:
import pandas as pd
import psycopg2
from sql_queries import create_table_queries, drop_table_queries

conn = psycopg2.connect(database="postgres", user='postgres', password='0908', host='127.0.0.1', port= '5432')
conn.set_session(autocommit=True)
cursor = conn.cursor()
cursor.execute('''DROP DATABASE IF EXISTS sparkifydb''')
cursor.execute('''CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0''')

conn.close()

conn = psycopg2.connect(database="sparkifydb", user='postgres', password='0908', host='127.0.0.1', port= '5432')
cursor = conn.cursor()

for command in drop_table_queries:
    cursor.execute(command)
    conn.commit()
    
for command in create_table_queries:
    cursor.execute(command)
    conn.commit()

cursor.close()
conn.close()

In [4]:
cursor.close()
conn.close()

In [5]:
import pandas as pd
import psycopg2
from sql_queries import create_table_queries, drop_table_queries

conn = psycopg2.connect(database="sparkifydb", user='postgres', password='0908', host='127.0.0.1', port= '5432')
cursor = conn.cursor()


In [6]:
pd.read_sql('select * from songplays', conn)

c:\Users\ASUS ZENBOOK\miniconda3\envs\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,16667,1541597788796,44,paid,None,None,285,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
1,16668,1541597979796,15,paid,None,None,221,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
2,16669,1541598039796,44,paid,None,None,285,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
3,18259,1542186737796,49,paid,None,None,568,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...
4,18260,1542186888796,58,paid,None,None,522,"Augusta-Richmond County, GA-SC","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
...,...,...,...,...,...,...,...,...,...
7765,16662,1541597130796,32,free,None,None,320,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
7766,16663,1541597170796,15,paid,None,None,221,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
7767,16664,1541597548796,15,paid,None,None,221,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
7768,16665,1541597551796,44,paid,None,None,285,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
